In [42]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report

In [43]:
data =pd.read_excel(r"C:\Users\yuvra\Downloads\transaction_anomalies_datasets.xlsx")
data.head(15)

,Transaction_ID,Transaction_Amount,Transaction_Volume,Average_Transaction_Amount,Frequency_of_Transactions,Time_Since_Last_Transaction,Day_of_Week,Time_of_Day,Age,Gender,Income,Account_Type
0,TX101,1298.864813,4.0,1237.027020,19.0,27.0,Wednesday,19:00:00,40.0,Male,5367215.0,Current
1,TX102,1344.203497,6.0,1337.765829,29.0,68.0,Saturday,22:00:00,61.0,Female,9173924.0,Savings
2,TX103,1339.952065,7.0,1340.210306,31.0,62.0,Saturday,17:00:00,73.0,Male,7217069.0,Savings
3,TX104,1320.956093,5.0,1349.496604,30.0,52.0,Tuesday,09:00:00,83.0,Female,81386232.0,Savings
4,TX105,1360.822254,5.0,1329.522480,41.0,78.0,Sunday,20:00:00,81.0,Male,71119030.0,Savings
5,TX106,1318.777241,4.0,1367.111026,32.0,45.0,Friday,14:00:00,86.0,Female,81149457.0,Savings
6,TX107,1289.987308,3.0,1368.948796,31.0,71.0,Thursday,20:00:00,54.0,Male,13742843.0,Savings
7,TX108,1293.317560,6.0,1359.889443,30.0,45.0,Tuesday,13:00:00,51.0,Male,91605889.0,Savings
8,TX109,1251.103035,8.0,1352.937667,35.0,77.0,Sunday,23:00:00,63.0,Male,21002283.0,Savings
9,TX110,1366.923329,7.0,1363.427366,44.0,73.0,Thursday,13:00:00,58.0,Female,1723247.0,Savings


In [44]:
data.columns

Index(['Transaction_ID', 'Transaction_Amount', 'Transaction_Volume',
       'Average_Transaction_Amount', 'Frequency_of_Transactions',
       'Time_Since_Last_Transaction', 'Day_of_Week', 'Time_of_Day', 'Age',
       'Gender', 'Income', 'Account_Type'],
      dtype='object')

In [45]:
data.isnull().sum()

Transaction_ID                 0
Transaction_Amount             0
Transaction_Volume             0
Average_Transaction_Amount     0
Frequency_of_Transactions      0
Time_Since_Last_Transaction    0
Day_of_Week                    0
Time_of_Day                    0
Age                            0
Gender                         0
Income                         0
Account_Type                   0
dtype: int64

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Transaction_ID               200 non-null    object 
 1   Transaction_Amount           200 non-null    float64
 2   Transaction_Volume           200 non-null    float64
 3   Average_Transaction_Amount   200 non-null    float64
 4   Frequency_of_Transactions    200 non-null    float64
 5   Time_Since_Last_Transaction  200 non-null    float64
 6   Day_of_Week                  200 non-null    object 
 7   Time_of_Day                  200 non-null    object 
 8   Age                          200 non-null    float64
 9   Gender                       200 non-null    object 
 10  Income                       200 non-null    float64
 11  Account_Type                 200 non-null    object 
dtypes: float64(7), object(5)
memory usage: 18.9+ KB


In [47]:
data.describe()

,Transaction_Amount,Transaction_Volume,Average_Transaction_Amount,Frequency_of_Transactions,Time_Since_Last_Transaction,Age,Income
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,2.000000e+02
mean,1324.547145,5.593422,1345.467149,50.701592,66.124021,77.451238,2.159813e+07
std,243.803399,2.959047,244.371360,41.406643,46.386672,39.062163,7.908469e+07
min,897.778849,1.000000,950.606023,5.000000,1.000000,19.000000,3.219590e+05
25%,1154.716821,3.000000,1191.427781,18.750000,26.750000,51.000000,1.106716e+06
50%,1293.813427,5.000000,1340.210306,31.000000,52.500000,68.000000,1.485440e+06
75%,1523.275597,8.000000,1557.833632,81.000000,111.000000,93.000000,8.425657e+06
max,1889.451798,13.000000,1833.868275,153.000000,185.000000,194.000000,9.213862e+08


In [48]:
fig_amount = px.histogram(data,x= 'Transaction_Amount',nbins=25,title='Distribution of Transaction')
fig_amount.show()

In [49]:
fig_box_amount = px.box(data, x='Account_Type',y='Transaction_Amount'
                       ,title ='Transaction Amount of Account')
fig_box_amount.show()

In [50]:
#Average Transaction AMount Vs Age
fig_scatter_avg_amount_age = px.scatter(data,x='Age',
                                       y='Average_Transaction_Amount',
                                       title='Average Transaction by Age',
                                       color='Account_Type'
                                    ,trendline='ols'
                                       )
fig_scatter_avg_amount_age.show()

In [51]:
# Count of Transactions by Day of the Week
fig_day_of_week = px.bar(data, x='Day_of_Week',
                         title='Count of Transactions by Day of the Week')
fig_day_of_week.show()

In [66]:
numeric_data = data.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix
correlation_matrix = numeric_data.corr()

fig_corr_heatmap = px.imshow(correlation_matrix,
                             labels=dict(x="Features", y="Features", color="Correlation"),
                             title='Correlation Heatmap')

# You can customize the color scale if needed
# Example:
# fig_corr_heatmap.update_layout(coloraxis_colorbar=dict(title='Custom Colorbar Title'))

fig_corr_heatmap.show()

In [53]:
# correlation_matrix = numeric_data.corr()
# fig_corr_heatmap = px.imshow(correlation_matrix, title='Correlation Heatmap')
# fig_corr_heatmap.show()

In [67]:
data.columns

Index(['Transaction_ID', 'Transaction_Amount', 'Transaction_Volume',
       'Average_Transaction_Amount', 'Frequency_of_Transactions',
       'Time_Since_Last_Transaction', 'Day_of_Week', 'Time_of_Day', 'Age',
       'Gender', 'Income', 'Account_Type', 'Is_Anomaly'],
      dtype='object')

In [68]:
data.head(5)

,Transaction_ID,Transaction_Amount,Transaction_Volume,Average_Transaction_Amount,Frequency_of_Transactions,Time_Since_Last_Transaction,Day_of_Week,Time_of_Day,Age,Gender,Income,Account_Type,Is_Anomaly
0,TX101,1298.864813,4.0,1237.027020,19.0,27.0,Wednesday,19:00:00,40.0,Male,5367215.0,Current,False
1,TX102,1344.203497,6.0,1337.765829,29.0,68.0,Saturday,22:00:00,61.0,Female,9173924.0,Savings,False
2,TX103,1339.952065,7.0,1340.210306,31.0,62.0,Saturday,17:00:00,73.0,Male,7217069.0,Savings,False
3,TX104,1320.956093,5.0,1349.496604,30.0,52.0,Tuesday,09:00:00,83.0,Female,81386232.0,Savings,False
4,TX105,1360.822254,5.0,1329.522480,41.0,78.0,Sunday,20:00:00,81.0,Male,71119030.0,Savings,False


In [56]:
#Calculate mean and standarad deviation o fTransaction Amount
mean_amount = data['Transaction_Amount'].mean()
std_amount = data['Transaction_Amount'].std()


In [57]:
# Define the anomaly threshold (2 standard deviations from the mean)
anomaly_threshold = mean_amount + 2 * std_amount

In [58]:
anomaly_threshold

1812.1539416295093

In [59]:
#flag anomalies
data['Is_Anomaly'] = data['Transaction_Amount'] >anomaly_threshold

In [60]:
data['Is_Anomaly'].sum()

1

In [61]:
fig_anomalies = px.scatter(data,x='Transaction_Amount',y='Average_Transaction_Amount',
                          color='Is_Anomaly',title='Anomalies in tRANSACTION Amount')
fig_anomalies.update_traces(marker = dict(size=12),
                           selector=dict(mode='markers',marker_size=1))
fig_anomalies.show()

In [62]:
# Calculate the number of anomalies
num_anomalies = data['Is_Anomaly'].sum()

# Calculate the total number of instances in the dataset
total_instances = data.shape[0]

# Calculate the ratio of anomalies
anomaly_ratio = num_anomalies / total_instances
print(anomaly_ratio)

0.005


In [63]:
total_instances 

200

In [81]:
relevant_features

['Transaction_Amount',
 'Average_Transaction_Amount',
 'Frequency_of_Transactions']

In [64]:
relevant_features = ['Transaction_Amount',
                     'Average_Transaction_Amount',
                     'Frequency_of_Transactions']

# Split data into features (X) and target variable (y)
X = data[relevant_features]
y = data['Is_Anomaly']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Isolation Forest model
model = IsolationForest(contamination=0.02, random_state=42)
model.fit(X_train)

C:\Users\yuvra\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

C:\Users\yuvra\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names



IsolationForest(contamination=0.02, random_state=42)

In [82]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest

# Assume you have a DataFrame 'data' with relevant features and the target variable 'Is_Anomaly'
relevant_features = ['Transaction_Amount', 'Average_Transaction_Amount', 'Frequency_of_Transactions']

# Split data into features (X) and target variable (y)
X = data[relevant_features]
y = data['Is_Anomaly']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set feature names for X_train
X_train.columns = relevant_features

# Train the Isolation Forest model
model = IsolationForest(contamination=0.02, random_state=42)
model.fit(X_train)


C:\Users\yuvra\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

C:\Users\yuvra\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names



IsolationForest(contamination=0.02, random_state=42)

In [84]:
# y_pred = model.predict(X_test)
# y_pred_binary = [1 if pred == -1 else 0 for pred in y_pred]
# report = classification_report(y_test, y_pred_binary,target_names=['Normal','Anomaly'])
# print(report)
# from sklearn.metrics import classification_report

y_pred_binary = ['Anomaly' if pred == -1 else 'Normal' for pred in y_pred]


In [87]:
y_test_str = y_test.astype(str)
report = classification_report(y_test_str, y_pred_binary, target_names=['Normal', 'Anomaly'])
print(report)


              precision    recall  f1-score   support

      Normal       0.00      0.00      0.00      40.0
     Anomaly       0.00      0.00      0.00       0.0

    accuracy                           0.00      40.0
   macro avg       0.00      0.00      0.00      40.0
weighted avg       0.00      0.00      0.00      40.0



C:\Users\yuvra\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\yuvra\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

C:\Users\yuvra\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

C:\Users\yuvra\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` param

In [89]:
# Relevant features used during training
relevant_features = ['Transaction_Amount', 'Average_Transaction_Amount', 'Frequency_of_Transactions']

# Get user inputs for features
user_inputs = []
for feature in relevant_features:
    user_input = float(input(f"Enter the value for '{feature}': "))
    user_inputs.append(user_input)

# Create a DataFrame from user inputs
user_df = pd.DataFrame([user_inputs], columns=relevant_features)

# Predict anomalies using the model
user_anomaly_pred = model.predict(user_df)

# Convert the prediction to binary value (0: normal, 1: anomaly)
user_anomaly_pred_binary = 1 if user_anomaly_pred == -1 else 0

if user_anomaly_pred_binary == 1:
    print("Anomaly detected: This transaction is flagged as an anomaly.")
else:
    print("No anomaly detected: This transaction is normal.")

Enter the value for 'Transaction_Amount': 2000
Enter the value for 'Average_Transaction_Amount': 1600
Enter the value for 'Frequency_of_Transactions': 15
Anomaly detected: This transaction is flagged as an anomaly.


C:\Users\yuvra\anaconda3\lib\site-packages\sklearn\utils\validation.py:623: FutureWarning:

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.

